In [ ]:
from dask_gateway import Gateway
url = 'http://128.110.223.7'
gateway = Gateway(url)
options = gateway.cluster_options()
# options.environment = dict(
#     DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1",
#     MALLOC_TRIM_THRESHOLD_="65536"
# )
cluster = gateway.new_cluster(options, shutdown_on_close=False)
cluster.scale(16)
client = cluster.get_client()
cluster

In [4]:
# coords =  [[[43.603765484965095,41.20946204327447],[43.603765484965095,39.85119150821595],[45.68577106349204,39.85119150821595],[45.68577106349204,41.20946204327447],[43.603765484965095,41.20946204327447]]]
coords =  [[[43.40590543408217,41.159840783028386],[43.40590543408217,40.35814947150769],[44.58924500968337,40.35814947150769],[44.58924500968337,41.159840783028386],[43.40590543408217,41.159840783028386]]]
def get_bbox(geometry):
    x_coordinates, y_coordinates = zip(*geometry[0])
    return [min(x_coordinates), min(y_coordinates), max(x_coordinates), max(y_coordinates)]
bbox = get_bbox(coords)
bbox

[43.40590543408217, 40.35814947150769, 44.58924500968337, 41.159840783028386]

In [67]:
from satsearch import Search
dataRepositoryURL = 'https://earth-search.aws.element84.com/v0/'
date_range = '2022-03-01/2022-03-30'
scene_cloud_tolerance = 50
bbox= [44.999772171826386, 40.555482279926174, 46.31618825636783, 41.55183566226867]
search = Search(bbox=bbox, datetime=date_range,
                        query={"eo:cloud_cover": {"lt": scene_cloud_tolerance}},
                        collections=["sentinel-s2-l2a-cogs"],
                        url=dataRepositoryURL)
items = search.items()
import stackstac
import numpy as np
epsg = 4326
chunk_size = 'auto' #'auto' #8192 #, 8192*2
stack = stackstac.stack(items[4], epsg=epsg, chunksize=chunk_size, assets=["B08", "B04"])
nir, red = stack.sel(band="B08"), stack.sel(band="B04")
graph = ((nir - red) / (nir + red))
graph = graph.mean('time')
stack
# stackstac.stack(items[3], assets=["B08"]).shape
# from skimage import io
# import matplotlib.pyplot as plt
# url = items[3].assets.get('thumbnail').get('href')
# # print(items[3].assets)
# image = io.imread(url)
# plt.imshow(image)
# plt.show()
# items[3].assets

<xarray.DataArray 'stackstac-0c84658fe5d954bdf0eff5f411705c78' (time: 1,
                                                                band: 2,
                                                                y: 11059,
                                                                x: 10972)>
dask.array<fetch_raster_window, shape=(1, 2, 11059, 10972), dtype=float64, chunksize=(1, 1, 4112, 4080), chunktype=numpy.ndarray>
Coordinates: (12/30)
  * time                          (time) datetime64[ns] 2022-03-29T07:48:11
    id                            (time) <U24 'S2B_38TPM_20220329_0_L2A'
  * band                          (band) <U3 'B08' 'B04'
  * x                             (x) float64 46.2 46.2 46.2 ... 47.55 47.55
  * y                             (y) float64 42.45 42.45 42.45 ... 41.44 41.44
    sentinel:latitude_band        <U1 'T'
    ...                            ...
    proj:shape                    object {10980}
    proj:transform                object {0, 600000, 1, 4700040, 10, -10}
    common_name                   (band) <U3 'nir' 'red'
    center_wavelength             (band) float64 0.8351 0.6645
    full_width_half_max           (band) float64 0.145 0.038
    epsg                          int64 4326
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(46.197261334719464, 41.4362...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 46.20|\n| 0.00,-0.00, 42.45|\n| 0.00, 0.00,...
    resolution_xy:  (0.00012331469623205749, 9.132925460875185e-05)

In [63]:
from dask.distributed import wait, progress
data = graph.compute()
# wait(data)

In [66]:
data.shape

(11059, 10972)

In [93]:
cluster.shutdown()

In [101]:
import ctypes
def trim_memory() -> int:
     libc = ctypes.CDLL("libc.so.6")
     return libc.malloc_trim(0)
client.run(trim_memory)

{'tls://192.168.5.16:44175': 1,
 'tls://192.168.5.248:37697': 1,
 'tls://192.168.84.233:39481': 1,
 'tls://192.168.84.240:37991': 1}

In [46]:
import gc
client.run(gc.collect)  # collect garbage on all workers

{'tls://192.168.5.129:45723': 61,
 'tls://192.168.5.130:36143': 52,
 'tls://192.168.5.137:33467': 108,
 'tls://192.168.5.141:37071': 183,
 'tls://192.168.5.144:44345': 52,
 'tls://192.168.5.145:41139': 52,
 'tls://192.168.5.146:43891': 52,
 'tls://192.168.5.147:40831': 62,
 'tls://192.168.5.148:43007': 62,
 'tls://192.168.84.121:45683': 62,
 'tls://192.168.84.124:42755': 111,
 'tls://192.168.84.129:38299': 349,
 'tls://192.168.84.130:40669': 230,
 'tls://192.168.84.132:34801': 79,
 'tls://192.168.84.134:44491': 62,
 'tls://192.168.84.136:35893': 121}

In [18]:
cluster.shutdown()

In [97]:
def calc(date_range = '2022-03-01/2022-04-01'): 
    dataRepositoryURL = 'https://earth-search.aws.element84.com/v0/'
    date_range = '2022-03-01/2022-04-01'
    scene_cloud_tolerance = 50
    search = Search(bbox=bbox, datetime=date_range,
                            query={"eo:cloud_cover": {"lt": scene_cloud_tolerance}},
                            collections=["sentinel-s2-l2a-cogs"],
                            url=dataRepositoryURL)
    items = search.items()
    print(len(items))
    import stackstac
    import numpy as np
    epsg = 4326
    chunk_size = (2048, 2048)
    stack = stackstac.stack(items, epsg=epsg, chunksize=chunk_size, assets=["B08", "B04"])
    nir, red = stack.sel(band="B08"), stack.sel(band="B04")
    graph = ((nir - red) / (nir + red))
    return graph.mean().compute()

calc()
calc('2022-04-01/2022-05-01')
calc('2022-05-01/2022-06-01')

54
54
54


<xarray.DataArray 'stackstac-8ea220878e2b615fec8a9abb262d9086' ()>
array(0.03474023)
Coordinates:
    sentinel:processing_baseline  <U5 '04.00'
    instruments                   <U3 'msi'
    sentinel:sequence             <U1 '0'
    sentinel:boa_offset_applied   bool True
    proj:epsg                     int64 32638
    view:off_nadir                int64 0
    gsd                           int64 10
    sentinel:valid_cloud_cover    bool True
    sentinel:utm_zone             int64 38
    constellation                 <U10 'sentinel-2'
    proj:shape                    object {10980}
    epsg                          int64 4326